<a href="https://colab.research.google.com/github/The-Supreme-Elemental/truck-drivers-logistics/blob/cj-branch/XGBMachineLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install lets_plot
!pip install kaggle

In [ ]:
# needed libraries for Regression models
import pandas as pd
from sklearn import tree
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, recall_score, precision_score, mean_squared_error
from sklearn.model_selection import cross_val_score,  train_test_split , KFold
from sklearn.preprocessing import StandardScaler, Normalizer
import lets_plot as lp
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import kagglehub
from kagglehub import KaggleDatasetAdapter
import os

In [ ]:
truck_path = kagglehub.dataset_download("ziya07/smart-logistics-supply-chain-dataset")
truck_df = pd.read_csv(f"{truck_path}/smart_logistics_dataset.csv")
truck_df = pd.DataFrame(truck_df)

In [ ]:
clean_truck_df = pd.DataFrame(truck_df)

clean_truck_df['Asset_ID'] = clean_truck_df['Asset_ID'].str.replace(r'\D+', '', regex=True).astype(int)
clean_truck_df['Traffic_Status'] = clean_truck_df['Traffic_Status'].astype('category')

clean_truck_df = pd.get_dummies(clean_truck_df, columns=['Timestamp'], drop_first=True)

X= clean_truck_df.drop(columns=['Shipment_Status','Logistics_Delay_Reason', 'Waiting_Time','Logistics_Delay', 'User_Transaction_Amount', 'User_Purchase_Frequency'])
y= clean_truck_df['Logistics_Delay']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify=y)


In [ ]:
cM = XGBClassifier(objective='binary:logistic', max_depth=1, n_estimators=100, random_state=42, enable_categorical=True )

In [ ]:
# Fitting the model
cM.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=True, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=1, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [ ]:
# Predicting the Test set results
y_pred = cM.predict(X_test)

In [ ]:
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(cM, X, y, cv=5, scoring='accuracy')
print("Cross-validation accuracy:", cv_scores.mean())

Cross-validation accuracy: 0.743


In [ ]:
# Accuracy, F1 Score, Recall score, Precision score
accuracy = accuracy_score(y_test, y_pred)*100
f1_score = f1_score(y_test, y_pred, average='weighted')
recall_score = recall_score(y_test, y_pred, average='weighted')
precision_score = precision_score(y_test, y_pred, average='weighted')

print('Accuracy: ' + str(round(accuracy, 2)) + ' %.')
print('F1 Score: ' + str(round(f1_score, 2)) + ' %.')
print('Recall Score: ' + str(round(recall_score, 2)) + ' %.')
print('Precision Score: ' + str(round(precision_score, 2)) + ' %.')

Accuracy: 73.0 %.
F1 Score: 0.73 %.
Recall Score: 0.73 %.
Precision Score: 0.76 %.


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

clean_truck_df = pd.DataFrame(truck_df)

clean_truck_df['Asset_ID'] = clean_truck_df['Asset_ID'].str.replace(r'\D+', '', regex=True).astype(int)
clean_truck_df['Traffic_Status'] = clean_truck_df['Traffic_Status'].astype('category')

clean_truck_df['Timestamp'] = pd.to_datetime(clean_truck_df['Timestamp'])
clean_truck_df['Hour'] = clean_truck_df['Timestamp'].dt.hour
clean_truck_df['Day'] = clean_truck_df['Timestamp'].dt.day
clean_truck_df['Month'] = clean_truck_df['Timestamp'].dt.month
clean_truck_df['Weekday'] = clean_truck_df['Timestamp'].dt.weekday

clean_truck_df = clean_truck_df.drop(columns=['Timestamp'])
clean_truck_df = pd.get_dummies(clean_truck_df, columns=['Traffic_Status'], drop_first=True)

X= clean_truck_df.drop(columns=['Shipment_Status','Logistics_Delay_Reason', 'Waiting_Time','Logistics_Delay', 'User_Transaction_Amount', 'User_Purchase_Frequency'])
y= clean_truck_df['Logistics_Delay']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify=y)


In [ ]:
param_grid = {
    'max_depth': [2, 3, 4],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200],
    'subsample': [0.7, 1.0],
    'colsample_bytree': [0.7, 1.0],
    'gamma': [0, 0.1, 0.5]
}

grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

In [ ]:
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_

print("Best Parameters:", grid_search.best_params_)
print("Cross-validation accuracy:", grid_search.best_score_)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: Using fork() can cause Polars to deadlock in the child process.
In addition, using fork() with Python in general is a recipe for mysterious
deadlocks and crashes.

The most likely reason you are seeing this error is because you are using the
multiprocessing module on Linux, which uses fork() by default. This will be
fixed in Python 3.14. Until then, you want to use the "spawn" context instead.

See https://docs.pola.rs/user-guide/misc/multiprocessing/ for details.

  pid = os.fork()


Best Parameters: {'colsample_bytree': 1.0, 'gamma': 0, 'learning_rate': 0.01, 'max_depth': 2, 'n_estimators': 200, 'subsample': 1.0}
Cross-validation accuracy: 0.7575


In [ ]:
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred) * 100
f1 = f1_score(y_test, y_pred, average='weighted') * 100
recall = recall_score(y_test, y_pred, average='weighted') * 100
precision = precision_score(y_test, y_pred, average='weighted') * 100

print('Accuracy: ' + str(round(accuracy, 2)) + ' %.')
print('F1 Score: ' + str(round(f1, 2)) + ' %.')
print('Recall Score: ' + str(round(recall, 2)) + ' %.')
print('Precision Score: ' + str(round(precision, 2)) + ' %.')

Accuracy: 77.0 %.
F1 Score: 76.54 %.
Recall Score: 77.0 %.
Precision Score: 84.24 %.
